## Лабораторная работа №2

Импортируем библиотеки

In [75]:

import pandas as pd
import plotly.express as px
import plotly
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact
import ipywidgets as widgets
from plotly.offline import iplot

Создаём Pandas DataFrame из файла IBM.csv

In [76]:
file_path = 'IBM.csv'
df = pd.read_csv(file_path)

Отрисуем график стоимости акций

In [77]:
fig = go.Figure(layout_yaxis_range=[0,np.max(df['Close'])*1.1])
fig.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Close'],
    mode="lines+markers"
))
fig.update_xaxes(title='Дата')
fig.update_yaxes(title='Стоимость')
fig.show()

### Сглаживание: Скользящее среднее

In [78]:
def moving_average(data, window_size):
    weights = np.repeat(1.0, window_size) / window_size
    full_data = np.pad(data, (window_size, ), 'edge')
    return np.convolve(full_data, weights, 'valid')

In [79]:
trace1 = go.Scatter(x=df['Date'], y=df['Close'], mode="lines+markers", name='Исходные данные')
window_size = 2
rolling_mean = moving_average(df['Close'], window_size)
trace2 = go.Scatter(x=df['Date'], y=rolling_mean, mode="lines+markers", name=f'Скользящее среднее {window_size}')
fig = go.FigureWidget(data=[trace1,trace2])
fig.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def response(x):
    window_size = int(x)
    rolling_mean = moving_average(df['Close'], window_size)
    fig["data"][1].y = rolling_mean
    fig["data"][1].name = f'Скользящее среднее {window_size}'
    

interact(response, x=widgets.IntSlider(min=2, max=len(df['Date']), step=1, value=2))
fig

interactive(children=(IntSlider(value=2, description='x', max=250, min=2), Output()), _dom_classes=('widget-in…

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': 'ae9a8d5b-5302-42da-9bf8-7e17f4d8baef',
              'x': array(['2022-09-19', '2022-09-20', '2022-09-21', ..., '2023-09-13',
                          '2023-09-14', '2023-09-15'], dtype=object),
              'y': array([127.730003, 126.300003, 124.93    , ..., 146.550003, 147.350006,
                          145.990005])},
             {'mode': 'lines+markers',
              'name': 'Скользящее среднее 2',
              'type': 'scatter',
              'uid': '760cec2b-47c2-41cb-9866-f3936caea6dd',
              'x': array(['2022-09-19', '2022-09-20', '2022-09-21', ..., '2023-09-13',
                          '2023-09-14', '2023-09-15'], dtype=object),
              'y': array([127.730003 , 127.730003 , 127.015003 , ..., 146.6700055, 145.990005 ,
                          145.990005 ])}],
    'layout': {'template': '...', 'xaxis': {'ti

### Сглаживание: Экспоненциальное скользящее среднее

In [80]:
def exponential_moving_average(data, alpha):
    ema = [data[0]]
    for i in range(1, len(data)):
        ema.append(alpha * data[i] + (1 - alpha) * ema[i - 1])
    return ema

In [81]:
trace1 = go.Scatter(x=df['Date'], y=df['Close'], mode="lines+markers", name='Исходные данные')
alpha = 0
rolling_mean = exponential_moving_average(df['Close'], alpha)
trace2 = go.Scatter(x=df['Date'], y=rolling_mean, mode="lines+markers", name=f'EMA {alpha}')
fig = go.FigureWidget(data=[trace1,trace2])
fig.update_layout(xaxis_title='Дата', yaxis_title='Стоимость')

def response(x):
    alpha = x
    rolling_mean = exponential_moving_average(df['Close'], alpha)
    fig["data"][1].y = rolling_mean
    fig["data"][1].name = f'EMA {alpha}'

interact(response, x=widgets.FloatSlider(min=0, max=1, step=0.01, value=0))
fig

interactive(children=(FloatSlider(value=0.0, description='x', max=1.0, step=0.01), Output()), _dom_classes=('w…

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Исходные данные',
              'type': 'scatter',
              'uid': '97462576-9a0a-48d5-b710-d106dfb5871a',
              'x': array(['2022-09-19', '2022-09-20', '2022-09-21', ..., '2023-09-13',
                          '2023-09-14', '2023-09-15'], dtype=object),
              'y': array([127.730003, 126.300003, 124.93    , ..., 146.550003, 147.350006,
                          145.990005])},
             {'mode': 'lines+markers',
              'name': 'EMA 0.0',
              'type': 'scatter',
              'uid': '1262e05e-4b1a-4f13-990e-d6e4ebb9ee83',
              'x': array(['2022-09-19', '2022-09-20', '2022-09-21', ..., '2023-09-13',
                          '2023-09-14', '2023-09-15'], dtype=object),
              'y': [127.730003, 127.730003, 127.730003, ..., 127.730003,
                    127.730003, 127.730003]}],
    'layout': {'template': '...', 'xaxis': {'title': {'text': 'Дата'}}, 'yaxis'